In [1]:
import os
import django

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "portfolio_management.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

from common.models import Brokers, Prices, Transactions, FX, Assets
from users.models import CustomUser
from utils import NAV_at_date, Irr, chart_dates, chart_labels, chart_colour, portfolio_at_date, calculate_security_nav, currency_format, calculate_portfolio_value
from datetime import date
from django.db.models import F, Sum

In [2]:
selected_brokers = broker_ids = [2]
effective_current_date = date.today()
currency_target = 'USD'
number_of_digits = 2

CustomUser.objects.get(id=2)

<CustomUser: Yaroslav>

In [3]:
security_id = 7
security = Assets.objects.get(id=security_id)
security.price_at_date(effective_current_date).price

quote = security.prices.filter(date__lte=effective_current_date).order_by('-date').first()
print(currency_target)
if currency_target is not None:
    quote.price = quote.price * FX.get_rate(security.currency, currency_target, effective_current_date)['FX']
quote

USD


<Prices: Long/short is at 23.972000000000 on 2020-12-01>

In [4]:
from django.db.models import Count

duplicates = (
            Prices.objects.values('date', 'security', 'price')
            .annotate(count_id=Count('id'))
            .filter(count_id__gt=1)
        )

total_duplicates = len(duplicates)
print((f"Found {total_duplicates} duplicate entries."))

for i, duplicate in enumerate(duplicates, start=1):
            # Keep one entry and delete the rest
            entries = Prices.objects.filter(
                date=duplicate['date'],
                security=duplicate['security'],
                price=duplicate['price']
            )
            entries_to_keep = entries.first()
            entries.exclude(id=entries_to_keep.id).delete()

            # Print status update
            print(f"Duplicates are being removed: {i} out of {total_duplicates}\r", end='', flush=True)
            
print("\nDuplicates removed.")


Found 0 duplicate entries.

Duplicates removed.


In [5]:
import datetime
from decimal import Decimal


def end_of_year_price_correction(user, year, broker_name, target_nav, asset_name):
    
    target_nav = round(Decimal(target_nav), 2)
    
    # Get the broker
    try:
        broker = Brokers.objects.get(name=broker_name)
    except Brokers.DoesNotExist:
        return {"error": f"Broker {broker_name} does not exist."}

    # Get the asset
    try:
        asset = Assets.objects.get(name=asset_name)
    except Assets.DoesNotExist:
        return {"error": f"Asset {asset_name} does not exist."}

    # Calculate end of year date
    end_of_year_date = datetime.date(year, 12, 31)

    # Fetch NAV at the end of the year
    # nav_at_end_of_year = get_nav_at_date(broker, end_of_year_date)
    nav_at_end_of_year = NAV_at_date(user.id, [broker.id], end_of_year_date, asset.currency, [])['Total NAV']
    if nav_at_end_of_year is None:
        return {"error": f"No NAV found for broker {broker_name} at the end of {year}."}

    # Fetch asset price and position at the end of the year
    price_at_end_of_year = asset.price_at_date(end_of_year_date)
    if not price_at_end_of_year:
        return {"error": f"No price found for asset {asset_name} at the end of {year}."}

    position_at_end_of_year = asset.position(end_of_year_date, [broker.id])

    # Calculate new price
    old_price = price_at_end_of_year.price
    new_price = old_price + ((target_nav - nav_at_end_of_year) / position_at_end_of_year)

    # Display information
    old_asset_value = round(Decimal(old_price * position_at_end_of_year), 2)
    new_asset_value = round(Decimal(new_price * position_at_end_of_year), 2)

    result = {
        "old_price": old_price,
        "new_price": new_price,
        "old_asset_value": old_asset_value,
        "new_asset_value": new_asset_value,
        "old_nav": nav_at_end_of_year,
        "target_nav": target_nav
    }

    print(f"Old Price: {result['old_price']}, New Price: {result['new_price']}")
    print(f"Old Asset Value: {result['old_asset_value']}, New Asset Value: {result['new_asset_value']}")
    print(f"Old NAV: {result['old_nav']}, Target NAV: {result['target_nav']}")

    # Ask for confirmation
    confirm = input("Do you want to update the price? (yes/no): ")

    if confirm.lower() == 'yes':
        # Update the price
        price_instance, created = Prices.objects.get_or_create(
            security=asset, date=end_of_year_date,
            defaults={'price': new_price}
        )
        if not created:
            price_instance.price = new_price
            price_instance.save()

        result["status"] = "Price updated successfully."
        print("New price is:", price_instance.price)
    else:
        result["status"] = "Price update canceled."

    return result

user = CustomUser.objects.filter(username='Yaroslav').first()
year = 2013
broker_name = 'UBS Pension'
target_nav = 29444.38
asset_name = 'Emerging Markets Equity Fund'

end_of_year_price_correction(user, year, broker_name, target_nav, asset_name)

Old Price: 0.936630, New Price: 0.9366287795154500008662569358
Old Asset Value: 7983.27, New Asset Value: 7983.26
Old NAV: 29444.390402671861, Target NAV: 29444.38


{'old_price': Decimal('0.936630'),
 'new_price': Decimal('0.9366287795154500008662569358'),
 'old_asset_value': Decimal('7983.27'),
 'new_asset_value': Decimal('7983.26'),
 'old_nav': Decimal('29444.390402671861'),
 'target_nav': Decimal('29444.38'),
 'status': 'Price update canceled.'}

In [6]:
from utils import import_asset_prices_from_csv

# file_path = "../Data/Pricing data from Galaxy.csv"
investor_id = 2

# import_asset_prices_from_csv(file_path, investor_id)

In [38]:
import pandas as pd
from fuzzywuzzy import process
from datetime import datetime
from django.db.models import Q
import sys

import constants

def parse_charles_stanley_transactions(file, currency, broker_id, investor_id):
    df = pd.read_excel(file, header=3)
    transactions = []
    security_cache = {}
    skipped_count = 0

    broker = Brokers.objects.get(id=broker_id)
    investor = CustomUser.objects.get(id=investor_id)

    quantity_field = Transactions._meta.get_field('quantity')
    quantity_decimal_places = quantity_field.decimal_places
    # price_field = Transactions._meta.get_field('price')
    price_decimal_places = 5

    def find_or_prompt_security(stock_description, investor, broker, security_cache):

        # Check if we have a cached mapping
        if stock_description in security_cache:
            return security_cache[stock_description]
        
        def find_best_matching_security(description):
            securities = Assets.objects.filter(
                Q(investor=investor) & Q(brokers=broker)
            )
            security_names = [security.name for security in securities]
            best_match = process.extractOne(description, security_names, score_cutoff=60)
            
            if best_match:
                match_name, match_score = best_match
                print(f"Potential match found: '{match_name}' (Similarity: {match_score}%)")
                sys.stdout.flush()
                user_confirm = input(f"Do you agree with this match for '{description}'? (yes/no/skip/exit): ").lower()
                
                if user_confirm == 'yes':
                    matched_security = securities.get(name=match_name)
                    security_cache[stock_description] = matched_security  # Update cache
                    print(f"Match confirmed and cached for future use.")
                    return matched_security
                elif user_confirm == 'exit':
                    raise KeyboardInterrupt("User requested to exit")
                elif user_confirm == 'skip':
                    print("Skipping this transaction.")
                    security_cache[stock_description] = None  # Cache the skip decision
                    return None
                else:
                    print("Match rejected.")
                    return None
            return None

        security = find_best_matching_security(stock_description)
        
        if security is None:
            print(f"No matching security found for '{stock_description}'.")
            sys.stdout.flush()  # Ensure print is displayed
            user_input = input("Please enter the correct security name, 'skip' to skip this transaction, or 'exit' to stop processing: ")
            
            if user_input.lower() == 'exit':
                raise KeyboardInterrupt("User requested to exit")
            elif user_input.lower() == 'skip':
                print("Skipping this transaction.")
                security_cache[stock_description] = None
            elif user_input:
                # Check for an exact match with the user input
                try:
                    security = Assets.objects.get(
                        Q(investor=investor) & Q(brokers=broker) & Q(name__iexact=user_input)
                    )
                    print(f"Security '{security.name}' found and selected.")
                     # Cache the user-defined mapping
                    security_cache[stock_description] = security
                except Assets.DoesNotExist:
                    print(f"No exact match found for '{user_input}'. This transaction will be skipped.")
                    security_cache[stock_description] = None
            else:
                print("No security name provided. This transaction will be skipped.")
                security_cache[stock_description] = None

        return security
    
    def transaction_exists(transaction_data):
        return Transactions.objects.filter(
            Q(investor=transaction_data['investor']) &
            Q(broker=transaction_data['broker']) &
            Q(security=transaction_data.get('security')) &
            Q(currency=transaction_data['currency']) &
            Q(type=transaction_data['type']) &
            Q(date=transaction_data['date']) &
            Q(quantity=transaction_data.get('quantity')) &
            Q(price=transaction_data.get('price')) &
            Q(cash_flow=transaction_data.get('cash_flow')) &
            Q(commission=transaction_data.get('commission'))
            ).exists()

    for index, row in df.iterrows():
        
        date_str = row['Date']

        # Check if there is any data in the current row
        if pd.notna(date_str):
            date = datetime.strptime(date_str, '%d-%b-%Y').date()
            description = row['Description']
            stock_description = row['Stock Description']
            price = row['Price']
            debit = row['Debit']
            credit = row['Credit']

            if description == '* BALANCE B/F *' or description == 'Cash Transfers ISA':
                print(f'Skipped: {description}')
                continue

            elif "Gross interest" in description:
                transaction_data = {
                    'investor': investor,
                    'broker': broker,
                    'currency': currency,
                    'type': constants.TRANSACTION_TYPE_INTEREST_INCOME,
                    'date': date,
                    'cash_flow': round(Decimal(str(credit)), 2),
                }

            elif description == 'Funds Platform Fee' or description == 'Govt Flat Rate Int Charge':
                transaction_data = {
                    'investor': investor,
                    'broker': broker,
                    'currency': currency,
                    'type': constants.TRANSACTION_TYPE_BROKER_COMMISSION,
                    'date': date,
                    'commission': round(-Decimal(str(debit)), 2),
                }

            elif description == 'Stocks & Shares Subs' or 'ISA Subscription' in description:
                transaction_data = {
                    'investor': investor,
                    'broker': broker,
                    'currency': currency,
                    'type': constants.TRANSACTION_TYPE_CASH_IN,
                    'date': date,
                    'cash_flow': Decimal(str(credit)),
                }

            elif 'Dividend' in description or 'Equalisation' in description or 'Tax Credit' in description:
                security = find_or_prompt_security(stock_description, investor, broker, security_cache)
                transaction_data = {
                    'investor': investor,
                    'broker': broker,
                    'security': security,
                    'currency': currency,
                    'type': constants.TRANSACTION_TYPE_DIVIDEND,
                    'date': date,
                    'cash_flow': Decimal(str(credit)),
                }

            elif pd.notna(stock_description):

                security = find_or_prompt_security(stock_description, investor, broker, security_cache)
                if security is None:
                    continue # Skip this transaction

                transaction_type = constants.TRANSACTION_TYPE_BUY if debit > 0 else constants.TRANSACTION_TYPE_SELL
                price = round(Decimal(str(price)), price_decimal_places)
                if transaction_type == constants.TRANSACTION_TYPE_BUY:
                    quantity = round(Decimal(str(debit)) / price, quantity_decimal_places)
                else:
                    quantity = round(-Decimal(str(credit)) / price, quantity_decimal_places)

                transaction_data = {
                    'investor': investor,
                    'broker': broker,
                    'security': security,
                    'currency': currency,
                    'type': transaction_type,
                    'date': date,
                    'quantity': quantity,
                    'price': price,
                }

            else:
                print(f'Skipped: {description}')
                continue  # Skip if not a recognized transaction type

            if transaction_exists(transaction_data):
                print(f"Skipping existing transaction: {transaction_data}")
                skipped_count += 1
                continue

            transactions.append(transaction_data)

    print(f"\nProcessed {len(transactions)} transactions.")
    print(f"Skipped {skipped_count} existing transactions.")

    if transactions:
        save_choice = input(f"Do you want to save these transactions for {broker.name}? (yes/no): ").lower()
        if save_choice == 'yes':
            Transactions.objects.bulk_create([Transactions(**data) for data in transactions])
            print("Transactions saved to the database.")
        else:
            print("Transactions were not saved to the database.")
    else:
        print("No transactions to save.")

    return transactions

file = "../Data/Statement_ISA_4681921_Generated_2158_08Aug.xlsx"
currency = 'GBP'
broker_id = 10
investor_id = 2

t = parse_charles_stanley_transactions(file, currency, broker_id, investor_id)

Potential match found: 'Legal & General Multi-Index' (Similarity: 86%)


Match rejected.
No matching security found for 'LEGAL & GENERAL UT L&G GBL TECH IDX TRUST I AC'.
Security 'Legal & General Global Technology Index' found and selected.
Skipped: Transfer Income to Trading
Skipped: Transfer Income to Trading
Skipped: Transfer Income to Trading
Skipped: Transfer Income to Trading
Skipped: Transfer Income to Trading
Skipped: Transfer Income to Trading
Skipped: Transfer Income to Trading
Skipped: Transfer Income to Trading
Skipped: Transfer to capital account
Skipped: Transfer from Div Ac
Skipped: Transfer Income to Trading
Skipped: Transfer Income to Trading
Skipped: Transfer Income to Trading
Skipped: Transfer Income to Trading
Skipped: Cash Transfers ISA
Skipped: Cash Transfers ISA
Skipped: Transfer Income to Trading
Skipped: Transfer Income to Trading
Skipped: Cash Transfers ISA
Skipped: Cash Transfers ISA
Skipped: Transfer to capital account
Skipped: Transfer from Div Ac
Potential match found: 'Vanguard FTSE 100' (Similarity: 86%)
Match rejected.
No ma

In [53]:
def calculate_buy_in_price(asset, date, currency=None, broker_id_list=None, start_date=None):
        """
        Calculates the buy-in price for the given date, currency, broker ID list, and start date.

        Args:
            date (datetime.date): The date for which to calculate the buy-in price.
            currency (str): The currency in which to calculate the buy-in price.
            broker_id_list (Optional[List[str]]): A list of broker IDs to filter the transactions by. Defaults to None.
            start_date (Optional[datetime.date]): The start date for the calculation. Defaults to None.

        Returns:
            float: The calculated buy-in price. Returns None if an error occurs.
        """
        
        # try:

        is_long_position = None

        transactions = asset.transactions.filter(
            quantity__isnull=False,  # Filter out transactions where quantity is not empty
            date__lte=date
        ).values('price', 'quantity', 'date', 'currency')

        if broker_id_list is not None:
            transactions = transactions.filter(broker_id__in=broker_id_list) 
        
        if not transactions:
            return None
        
        

        # Step 2: Get latest entry date
        entry_date = asset.entry_dates(date, broker_id_list)[-1]

        if start_date and start_date > entry_date:
            entry_date = start_date

            transactions = transactions.filter(date__gte=entry_date)
            position = asset.position(entry_date, broker_id_list)
            if position != 0:
                transactions = [{
                    'price': asset.price_at_date(entry_date).price,
                    'quantity': position,
                    'date': entry_date,
                    'currency': asset.currency,
                }] + list(transactions)
                is_long_position = position > 0
        else:
            transactions = transactions.filter(date__gte=entry_date)

        if is_long_position is None:
            first_transaction = transactions.order_by('date').first()
            if first_transaction:
                is_long_position = first_transaction['quantity'] > 0

        # Step 3: Amend the calculation method. For every entry transaction buy-in price is the weighted average of previous buy-in price and price for the current transaction.
        value_entry = Decimal(0)
        quantity_entry = Decimal(0)
        previous_entry_price = Decimal(0)

        print(transactions)

        for transaction in transactions:
            
            if currency is not None:
                fx_rate = FX.get_rate(transaction['currency'], currency, transaction['date'])['FX']
            else:
                fx_rate = 1

            if fx_rate:
                current_price = transaction['price'] * fx_rate
                weight_current = transaction['quantity']

                # Calculate entry price
                previous_entry_price = value_entry / quantity_entry if quantity_entry != 0 else 0
                weight_entry_previous = quantity_entry
                # If it's a long position and the quantity is positive, or if it's a short position and the quantity is negative, use the current price. Otherwise, use the previous buy-in price.
                entry_price = current_price if (is_long_position and transaction['quantity'] > 0) or (not is_long_position and transaction['quantity'] < 0) else previous_entry_price
                
                if (weight_entry_previous + weight_current) == 0:
                    entry_price = previous_entry_price
                else:
                    entry_price = (previous_entry_price * weight_entry_previous + entry_price * weight_current) / (weight_entry_previous + weight_current)
                quantity_entry += transaction['quantity']
                value_entry = entry_price * quantity_entry

                print(quantity_entry, value_entry, entry_price)

        return round(Decimal(value_entry / quantity_entry), 6) if quantity_entry else previous_entry_price

security = Assets.objects.get(id=43)
calculate_buy_in_price(security, date(2024,8,1), 'GBP')

<QuerySet [{'price': Decimal('3.902980'), 'quantity': Decimal('-17.894019'), 'date': datetime.date(2021, 9, 10), 'currency': 'GBP'}, {'price': Decimal('3.363980'), 'quantity': Decimal('445.900392'), 'date': datetime.date(2017, 8, 10), 'currency': 'GBP'}, {'price': Decimal('2.962970'), 'quantity': Decimal('337.499198'), 'date': datetime.date(2015, 1, 26), 'currency': 'GBP'}]>
-17.894019 0.000000 -0
428.006373 1500.000000680160000000000000 3.504620714327914925696678820
765.505571 2499.999999378220000000000000 3.265815552605847724131063182


Decimal('3.265816')